In [ ]:
# In order to test this file, you need to first launch multiple instances of SRC, with namespace
# ws1, ws2, ws3 ...

import sys
import os
import gymnasium as gym
from gymnasium import spaces
import numpy as np
import time
import re
import threading
import time
import numpy as np
from PyKDL import Frame, Rotation, Vector
from gym.spaces.box import Box
from surgical_robotics_challenge.psm_arm import PSM
from surgical_robotics_challenge.ecm_arm import ECM
from utils.scene import Scene
from surgical_robotics_challenge.simulation_manager import SimulationManager
from utils.needle_initialization import NeedleInitialization
from utils.needle_kinematics_v2 import NeedleKinematics_v2
from surgical_robotics_challenge.kinematics.psmFK import *


In [ ]:
# Define the controlling API
def Frame2Vec(goal_frame,bound = True):
    X_goal = goal_frame.p.x()
    Y_goal = goal_frame.p.y()
    Z_goal = goal_frame.p.z()
    rot_goal = goal_frame.M
    roll_goal,pitch_goal,yaw_goal  = rot_goal.GetRPY()
    if bound:
        if (roll_goal <= np.deg2rad(-360)):
            roll_goal += 2*np.pi
        elif (roll_goal > np.deg2rad(0)):
            roll_goal -= 2*np.pi
    array_goal = np.array([X_goal,Y_goal,Z_goal,roll_goal,pitch_goal,yaw_goal],dtype=np.float32)
    return array_goal

def needle_randomization(needle):
    origin_p = Vector( -0.0207937, 0.0562045, 0.0711726)
    origin_rz = 0.0

    random_x = np.random.uniform(-0.003, 0.003)
    random_y = np.random.uniform(-0.03, 0.01)
    random_rz = np.random.uniform(-np.pi/4,np.pi/4)

    origin_p[0] += random_x
    origin_p[1] += random_y
    origin_rz += random_rz

    new_rot = Rotation(np.cos(origin_rz),-np.sin(origin_rz),0,
                        np.sin(origin_rz),np.cos(origin_rz),0,
                        0.0,0.0,1.0)
    
    needle_pos_new = Frame(new_rot,origin_p)


    needle.needle.set_pose(needle_pos_new)

def needle_goal_evaluator(needle_kin, psm_arm, lift_height=0.007, psm_idx=2, deg_angle = None):

    if deg_angle is None:
        grasp_in_World = needle_kin.get_bm_pose()

    else:
        grasp_in_World = needle_kin.get_pose_angle(deg_angle)

    lift_in_grasp_rot = Rotation(1, 0, 0,
                                0, 1, 0,
                                0, 0, 1)    
    lift_in_grasp_trans = Vector(0,0,lift_height)
    lift_in_grasp = Frame(lift_in_grasp_rot,lift_in_grasp_trans)

    if psm_idx == 2:
        gripper_in_lift_rot = Rotation(0, -1, 0,
                                        -1, 0, 0,
                                        0, 0, -1)
    else:
        gripper_in_lift_rot = Rotation(0, 1, 0,
                                        1, 0, 0,
                                        0, 0, -1)           

    gripper_in_lift_trans = Vector(0.0,0.0,0.0)
    gripper_in_lift = Frame(gripper_in_lift_rot,gripper_in_lift_trans)

    gripper_in_world = grasp_in_World*lift_in_grasp*gripper_in_lift
    gripper_in_base = psm_arm.get_T_w_b()*gripper_in_world
    

    array_goal_base = Frame2Vec(gripper_in_base)
    array_goal_base = np.append(array_goal_base,0.0)
    return array_goal_base

def psm_step(obs,psm_arm):
    X= obs[0]
    Y = obs[1]
    Z = obs[2]
    Roll = obs[3]
    Pitch = obs[4]
    Yaw = obs[5]
    Jaw_angle = obs[6]
    T_goal = Frame(Rotation.RPY(Roll,Pitch,Yaw),Vector(X,Y,Z))
    psm_arm.servo_cp(T_goal)
    psm_arm.set_jaw_angle(Jaw_angle)  


def psm_step_move(obs,psm_arm,execute_time=0.5):
    X= obs[0]
    Y = obs[1]
    Z = obs[2]
    Roll = obs[3]
    Pitch = obs[4]
    Yaw = obs[5]
    Jaw_angle = obs[6]
    T_goal = Frame(Rotation.RPY(Roll,Pitch,Yaw),Vector(X,Y,Z))
    psm_arm.move_cp(T_goal,execute_time)
    psm_arm.set_jaw_angle(Jaw_angle)


In [ ]:
# Env initialization
ns_list = ['ws1','ws2','ws3','ws4']

scene_list = []
psm1_list = []
psm2_list = []
needle_list = []
needle_kin_list = []

simulation_manager = SimulationManager('src_client')
world_handle = simulation_manager.get_world_handle()
init_psm1 = np.array([ 0.04629208,0.00752399,-0.08173992,-3.598019,-0.05762508,1.2738742,0.8],dtype=np.float32)
init_psm2 = np.array([-0.03721037,  0.01213105, -0.08036895, -2.7039163, 0.07693613, 2.0361109, 0.8],dtype=np.float32)

for i in range(0,len(ns_list)):
    ns = ns_list[i]
    scene_list.append(Scene(simulation_manager,ns))
    psm1_list.append(PSM(simulation_manager, ns+'/ambf/env/psm1',add_joint_errors=False))
    psm2_list.append(PSM(simulation_manager, ns+'/ambf/env/psm2',add_joint_errors=False))
    needle_list.append(NeedleInitialization(simulation_manager,ns))
    needle_kin_list.append(NeedleKinematics_v2(ns))
    psm_step(init_psm1,psm1_list[i])   
    psm_step(init_psm2,psm2_list[i])




In [ ]:
needle_randomization(needle_list[0])
needle_randomization(needle_list[1])
needle_randomization(needle_list[2])

In [ ]:
# Multi-env control
def needle_reaching(thread_id, init_psm1, init_psm2, psm1_list, psm2_list, needle_kin_list):
    steps = 1
    while steps < 10:
        needle_randomization(needle_list[thread_id])
        psm_step(init_psm1, psm1_list)
        psm_step(init_psm2, psm2_list)
        time.sleep(1.0)
        flag = 1
        mid_goal = None
        episode_transitions = []
        goal_obs = needle_goal_evaluator(needle_kin_list, psm2_list, lift_height=0.007, psm_idx=2, deg_angle=None)
        temp_goal = np.copy(goal_obs)
        temp_goal[-1] = 0.8
        execute_time = np.random.uniform(1.0, 3.0)
        psm_step_move(temp_goal, psm2_list, execute_time)
        psm2_list.set_jaw_angle(0.8)
        time.sleep(2.5)
        psm_step_move(goal_obs, psm2_list, 0.2)
        psm2_list.set_jaw_angle(0.0)
        time.sleep(0.3)
        
        print(f"Thread {thread_id}: Completed step {steps}, iteration {i}")
        
        steps += 1


threads = []
for i in range(3):
    thread = threading.Thread(target=needle_reaching, 
                                args=(i, init_psm1, init_psm2, 
                                    psm1_list[i], psm2_list[i], needle_kin_list[i]))
    threads.append(thread)
    thread.start()

for thread in threads:
    thread.join()

print("All trajectories collected.")